In [3]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pyupbit
import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [4]:
class Data_preprocess() :
    def __init__(self, ticker, interval, to, count, norm) :
        self.norm = {"minmax" : self.MinMax,
                    "stand" : self.standarization,
                    "diff" : self.diff}
        
        self.data, self.label, self.dataset = self.preprocess(pyupbit.get_ohlcv(ticker=ticker, interval=interval, to=to, count=count),
                                                             normalization=norm)
        
    def MinMax(self, df) :
        norm = MinMaxScaler()
        norm_dataset = norm.fit_transform(df)
        return pd.DataFrame(norm_dataset, columns=list(df.columns))
    
    def standarization(self, df) :
        for col in df:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
        return df
    
    def diff(self, df) :
        for col in (df):
            log_y = np.log(df[col])
            df[col][1:] = np.diff(log_y)
        return df[1:]
    

    def add_label(self, dataset_df) :
        after10 = np.zeros_like(dataset_df['close'])
        for i in range(len(dataset_df['close']) - 1) :
            if dataset_df['close'][i + 1] > dataset_df['close'][i] :
                after10[i] = 1
            else : 
                after10[i] = 0
            
        return pd.DataFrame(after10,columns=['label'])
    
    
    def drop_feature(self, dataset_df) :
        # index(시간) 제거
        dataset_df = dataset_df.reset_index(drop=True)
        # value 제거
#         dataset_df = dataset_df.drop(columns=['value'])
        return dataset_df
    
    
    def add_avgPrice(self, dataset_df) :
        return (dataset_df['high'] + dataset_df['low'] + 
                dataset_df['open'] + dataset_df['close']) // 4
       
    
    def preprocess(self, dataset, normalization) :
        
        # drop feature
        dataset_df = self.drop_feature(dataset)
        
        # avg_price 추가
        dataset_df['avg_price'] = self.add_avgPrice(dataset_df)
        
        # label 추가
        if normalization == "diff" :
            label = self.add_label(dataset_df)[1:-1]
        else :
            label = self.add_label(dataset_df)[:-1]
        
        norm_df = self.norm[normalization](dataset_df.copy())[:-1]
        
        return norm_df, label, dataset_df[:-1]
        
        

In [5]:
ticker = 'KRW-BTC'
interval ='minute10'
to = f'2022-03-29 00:00'
count = 144000

processed_data =  Data_preprocess(ticker, interval, to, count, "stand")
display(processed_data.data)
display(processed_data.label)
display(processed_data.dataset)

,open,high,low,close,volume,value,avg_price
0,-0.733174,-0.733647,-0.733796,-0.732783,1.572948,0.171843,-0.733353
1,-0.732785,-0.733259,-0.733189,-0.732654,0.184025,-0.284567,-0.732975
2,-0.732612,-0.732007,-0.731846,-0.732654,0.237146,-0.266384,-0.732283
3,-0.732655,-0.730497,-0.731412,-0.729842,-0.065221,-0.365857,-0.731104
4,-0.729844,-0.730972,-0.730762,-0.731745,0.128170,-0.302029,-0.730834
...,...,...,...,...,...,...,...
143994,1.106060,1.101068,1.098904,1.096201,1.106177,1.728938,1.100564
143995,1.096287,1.095673,1.100941,1.095898,0.054821,0.381045,1.097202
143996,1.095854,1.094422,1.094527,1.099488,0.284602,0.673258,1.096078
143997,1.099444,1.096105,1.100118,1.095380,-0.188111,0.069847,1.097764


,label
0,1.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
143994,0.0
143995,1.0
143996,0.0
143997,1.0


,open,high,low,close,volume,value,avg_price
0,14655000.0,14672000.0,14612000.0,14664000.0,174.284892,2.551033e+09,14650750.0
1,14664000.0,14681000.0,14626000.0,14667000.0,67.288968,9.860506e+08,14659500.0
2,14668000.0,14710000.0,14657000.0,14667000.0,71.381157,1.048397e+09,14675500.0
3,14667000.0,14745000.0,14667000.0,14732000.0,48.088255,7.073136e+08,14702750.0
4,14732000.0,14734000.0,14682000.0,14688000.0,62.986226,9.261731e+08,14709000.0
...,...,...,...,...,...,...,...
143994,57184000.0,57185000.0,56901000.0,56955000.0,138.327126,7.890153e+09,57056250.0
143995,56958000.0,57060000.0,56948000.0,56948000.0,57.335712,3.268367e+09,56978500.0
143996,56948000.0,57031000.0,56800000.0,57031000.0,75.036941,4.270333e+09,56952500.0
143997,57031000.0,57070000.0,56929000.0,56936000.0,38.621467,2.201300e+09,56991500.0


In [6]:
processed_data.dataset.to_csv('../data/144000_data.csv')
processed_data.label.to_csv('../data/144000_label.csv')


In [7]:
csv_data = pd.read_csv('../data/144000_data.csv').drop(columns=['Unnamed: 0'])
csv_label = pd.read_csv('../data/144000_label.csv').drop(columns=['Unnamed: 0'])

In [6]:
def EachColumnMinMax(df) :
    norm = MinMaxScaler()
    df = norm.fit_transform(np.array(df).reshape(-1, 1)).squeeze(1)
    return df

def EachColumnStand(df) :
    df = (df - df.mean()) / df.std()
    return df

def EachColumnDiff(df) :
    log_y = np.log(df)
    df[1:] = np.diff(log_y)
    return df[1:]


def WindowDataGenerator(df_data, df_label, window_size, stride, norm) :
    
    if norm == "diff" :
        num_sample = ((df_data.shape[0] - 1) - window_size) // stride + 1
        data = np.zeros([window_size - 1, df_data.shape[1], num_sample])
    else : 
        num_sample = (df_data.shape[0] - window_size) // stride + 1
        data = np.zeros([window_size, df_data.shape[1], num_sample])

    
    labels = np.zeros([num_sample])

    for i in tqdm(range(num_sample)) :
        tmp = {}
        data_start = stride * i
        data_end = data_start + window_size
        
        for col in df_data.columns :
            if norm == "diff" :
                tmp[col] = EachColumnDiff(df_data[col][data_start : data_end])
            elif norm == "stand" :
                tmp[col] = EachColumnStand(df_data[col][data_start : data_end])
            elif norm == "minmax" :
                tmp[col] = EachColumnMinMax(df_data[col][data_start : data_end])

        data[:, :, i] = pd.DataFrame(tmp).values
        labels[i] = df_label.values[data_end - 1]
        
        
    data = data.transpose((2, 0, 1))
    print("dataset shape ==== ",data.shape)
    
    # data shape (80, 600, 6), label shape (80,)
    return torch.Tensor(data), torch.Tensor(labels)

In [7]:
# display(csv_data[1:-1].shape)

# a,b = WindowDataGenerator(csv_data[:-1], csv_label, 24 * 6, 117, "diff")
# print(a.shape)
# print(b.shape)

In [8]:
class Transformer2FC(nn.Module) :
    def __init__(self, input_shape, d_model, n_head, num_layer, dropout, num_class=2):
        super(Transformer2FC, self).__init__()
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layer)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        self.Encoder = nn.Sequential(
            nn.Linear(input_shape[1], d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, d_model)
        )
        
        self.linear = nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, 1)
        )

        self.linear2 = nn.Sequential(
            nn.Linear(input_shape[0], input_shape[0]//2),
            nn.ReLU(),
            nn.Linear(input_shape[0]//2, num_class)
        )
        
#         self.sigmoid = nn.Softmax()
    
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    def forward(self, x, masked_x) :
        # (batch, data, dim)
#         print("input shape : ", x.shape)
        x = self.Encoder(x)
#         print("Encoder shape : ", x.shape)
        x = self.pos_encoder(x)
#         print("pos_encoder shape : ", x.shape)
#         print("masked_x shape : ", masked_x.shape)
        x = self.transformer_encoder(x.transpose(0,1), masked_x).transpose(0, 1)
#         print("transformer_encoder shape : ", x.shape)
        x = self.linear(x)
#         print("linear shape : ", x.shape)
        x = x.squeeze(2)
#         print("squeeze shape : ", x.shape)
        x = self.linear2(x)
#         print("linear2 shape : ", x.shape)
        x = x.squeeze(1)
#         print("squeeze shape : ", x.shape)
        return x

class PositionalEncoding(nn.Module) :
    def __init__(self, d_model, dropout=0.1, max_len=5000) :
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x) :
        x = x + self.pe[:x.size(0), :]

        return self.dropout(x)

def get_attention_mask(x) :
    mask = torch.eq(x, 0)
    return mask

In [9]:
class WindowDataset(Dataset) :
    def __init__(self, df_data, df_label, window_size=144, stride=6, norm="diff") :
        self.data, self.label = self.WindowDataGenerator(df_data, df_label, window_size, stride, norm)
        
    def __getitem__(self, i) :
        return self.data[i], self.label[i]
                
    def __len__(self) :
        assert len(self.data) == len(self.label), "data와 label의 길이가 다름"
        return len(self.data)
        
    
    def EachColumnMinMax(self, df) :
        norm = MinMaxScaler()
        df = norm.fit_transform(np.array(df).reshape(-1, 1)).squeeze(1)
        return df

    def EachColumnStand(self, df) :
        df = (df - df.mean()) / df.std()
        return df

    def EachColumnDiff(self, df) :
        log_y = np.log(df)
        df[1:] = np.diff(log_y)
        return df[1:]


    def WindowDataGenerator(self, df_data, df_label, window_size, stride, norm) :
        if norm == "diff" :
            num_sample = ((df_data.shape[0] - 1) - window_size) // stride + 1
            data = np.zeros([window_size - 1, df_data.shape[1], num_sample])
        else : 
            num_sample = (df_data.shape[0] - window_size) // stride + 1
            data = np.zeros([window_size, df_data.shape[1], num_sample])
            
        labels = np.zeros([num_sample])

        for i in tqdm(range(num_sample)) :
            tmp = {}
            data_start = stride * i
            data_end = data_start + window_size
            for col in df_data.columns :
                if norm == "diff" :
                    tmp[col] = self.EachColumnDiff(df_data[col][data_start : data_end].copy())
                elif norm == "stand" :
                    tmp[col] = self.EachColumnStand(df_data[col][data_start : data_end].copy())
                elif norm == "minmax" :
                    tmp[col] = self.EachColumnMinMax(df_data[col][data_start : data_end].copy())

#                 tmp[col] = self.EachColumnMinMax(df_data.loc[data_start : data_end - 1, col])

            data[:, :, i] = pd.DataFrame(tmp).values
            labels[i] = df_label.values[data_end - 1]

        data = data.transpose((2, 0, 1))
        print("dataset shape ==== ",data.shape)

        # data shape (80, 600, 6), label shape (80,)
        return torch.Tensor(data), torch.Tensor(labels)

In [10]:
device = torch.device("cuda:0")
lr = 1e-4
epochs = 300
window_size = 24 * 6
window_stride = 12
feature_len = 7
batch_size = 128
num_class = 1

model = Transformer2FC(input_shape=(window_size, feature_len), 
                       d_model=512, 
                       n_head=8, 
                       num_layer=4, 
                       dropout=0.3, 
                       num_class = num_class).to(device)
# model = MLSTMfcn(max_seq_len=window_size, num_features=feature_len).to(device)
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.HuberLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
dataset = WindowDataset(csv_data, 
                        csv_label,
                       window_size = window_size, 
                       stride = window_stride, norm="stand")

dataloader = DataLoader(dataset, batch_size=batch_size)

 56%|██████████████████████████████████████████▎                                 | 6676/11988 [00:31<00:27, 193.55it/s]

In [ ]:
def cal_accuracy(pred, label, threshold=0) :
    acc = torch.zeros(pred.shape[0])
    acc[pred > threshold] = 1
    acc[pred < threshold] = 0
    score = [1 if acc[i] == label[i] else 0 for i in range(pred.shape[0])]
    return sum(score) / pred.shape[0]

In [ ]:
# progress = tqdm(range(epoch))

model.train()
for e in range(epochs) :
    batchloss = 0.0
    batchacc= 0
    tqdm_dataloader = tqdm(dataloader)
    for batch, (data, label) in enumerate(tqdm_dataloader, start=1) :
        optim.zero_grad()
        src_mask = model.generate_square_subsequent_mask(data.shape[1]).to(device)      
        
        pred = model(data.to(device), src_mask)
        
        loss = criterion(pred, label.to(device, dtype=torch.float64))
#         print(data.shape)
#         print(pred.shape)
        loss.backward()
        optim.step()
        
        score = cal_accuracy(pred.cpu(), label.cpu())
        
        batchloss += loss
        batchacc += score        
        
        tqdm_dataloader.set_description(f"Epoch[{e}/{epochs}]   mean loss : {batchloss/batch:0.7f}   mean acc : {batchacc/batch:0.7f}")
#         progress.set_description("loss : {:0.6f}".format(loss.cpu().item()))
        
        
#     print(f"Epoch : [{i} / {epoch}] Loss : {round((batchloss/len(dataloader)).item(), 4)} Acc : {round((batchacc/len(dataloader)), 4)}")
#     print(pred)
#     print(label)
#     print(f"Epoch : [{i} / {epoch}] Loss : {round((batchloss/len(dataloader)).item(), 4)}")
    
torch.save(model.state_dict(), f'./{epochs}E_stand_model.pt')

# 학습 성공 config

In [ ]:
ticker = 'KRW-BTC'
interval ='minute10'
to = f'2022-03-29 00:00'
count = 14400

lr = 1e-4
epochs = 300
window_size = 24 * 6
window_stride = 12
feature_len = 7
batch_size = 128
num_class = 1

norm = "stand"